In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
import numpy as np
from scipy.stats import pearsonr
import seaborn as sns
import datetime
import requests
from localcider.sequenceParameters import SequenceParameters
from Bio import SeqIO
from io import StringIO
from scipy import stats
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import statsmodels.stats.multitest
import sys
import matplotlib as mpl
from scipy.stats import linregress
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from sklearn import linear_model
import random
from matplotlib.colors import LinearSegmentedColormap
from Bio.Align import PairwiseAligner
from scipy.cluster.hierarchy import linkage, dendrogram
from Bio.Align import PairwiseAligner
from Bio import pairwise2
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
file_path = 'RBPs.xlsx'  # Replace with the path to your file
df = pd.read_excel(file_path, header=0, skiprows=[1])

In [5]:
genemap_df = pd.read_csv('hgnc_complete_set_2024-10-01.txt', delimiter='\t')

C:\Users\faris\AppData\Local\Temp\ipykernel_23996\4027190928.py:1: DtypeWarning: Columns (32,34,38,40,50) have mixed types. Specify dtype option on import or set low_memory=False.
  genemap_df = pd.read_csv('hgnc_complete_set_2024-10-01.txt', delimiter='\t')


In [ ]:
def get_domain_annotations(uniprot_id):
    """Fetches domain annotations using the UniProt ID from the UniProt's features endpoint."""
    base_url = f"https://www.ebi.ac.uk/proteins/api/features/{uniprot_id}"
    response = requests.get(base_url, headers={"Accept": "application/json"})
    if response.ok:
        domains = {}
        for feat in response.json()['features']:
            if feat['type'] == 'DOMAIN':
                domains[ feat['description'] ] = {}
                domains[ feat['description'] ]['begin'] = feat['begin']
                domains[ feat['description'] ]['end'] = feat['end']
                #print( feat ) 
                #feat['description']
                #feat['begin']
                #feat['end']
        return domains
#        return response.json()
    
    else:
        raise ValueError(f"Error fetching domain annotations for {uniprot_id}")
        

def get_protein_sequence_mapped(gene_name ):
    
    uniprot_ids = genemap_df[genemap_df['ensembl_gene_id']==gene_name]['uniprot_ids'].tolist()

    if len( uniprot_ids ) < 1: 
        print( gene_name )
        return '', ''
        
    uniprot_id = uniprot_ids[0]
    # URL to retrieve the sequence in FASTA format
    url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.fasta"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        fasta_data = response.text
        # Parse the FASTA format and extract the sequence
        lines = fasta_data.splitlines()
        protein_sequence = ''.join(lines[1:])  # Join all lines except the header
        return protein_sequence, uniprot_id
    else:
        raise ValueError(f"Protein sequence not found for UniProt ID: {uniprot_id}")


gene_name_to_protein_seq = {}
gene_name_to_uniprot_id = {}
gene_name_to_domains = {}
for index, row in df.iterrows():
    gene_name = row['Name']
    ensembl_gene_id = row['ID']
    ensembl_gene_id_map = ''
    if ';' in ensembl_gene_id:
        for gid in ensembl_gene_id.split(';'):
            if gid in genemap_df['ensembl_gene_id'].tolist():
                ensembl_gene_id_map = gid
                break
                
    else:
        ensembl_gene_id_map = ensembl_gene_id

    if ensembl_gene_id_map == '':
        print( f"didn't find mapped id for {row['ID']}" )
    
    try:
        protein_sequence, uniprot_id = get_protein_sequence_mapped(gene_name=ensembl_gene_id_map)
        gene_name_to_protein_seq[ gene_name ] = protein_sequence
        gene_name_to_uniprot_id[ gene_name ] = uniprot_id
        if uniprot_id == '':
            gene_name_to_domains[ gene_name ] = {}
        else:
            domains = get_domain_annotations( uniprot_id )
            gene_name_to_domains[ gene_name ] = domains
    except ValueError as e:
        print(f"Error for Gene: {gene_name}: {e}")
        
    if (index % 100) == 0: 
        print( index )

In [ ]:
# manually add the missing proteins
uniprot_id = 'P42771'
gene_name = 'CDKN2A'
url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.fasta"

response = requests.get(url)

if response.status_code == 200:
    fasta_data = response.text
    # Parse the FASTA format and extract the sequence
    lines = fasta_data.splitlines()
    protein_sequence = ''.join(lines[1:])  # Join all lines except the header
gene_name_to_protein_seq[ gene_name ] = protein_sequence
gene_name_to_uniprot_id[ gene_name ] = uniprot_id
domains = get_domain_annotations( uniprot_id )
gene_name_to_domains[ gene_name ] = domains

################################################
uniprot_id = 'Q9UPN3'
gene_name = 'MACF1'
url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.fasta"

response = requests.get(url)

if response.status_code == 200:
    fasta_data = response.text
    # Parse the FASTA format and extract the sequence
    lines = fasta_data.splitlines()
    protein_sequence = ''.join(lines[1:])  # Join all lines except the header
gene_name_to_protein_seq[ gene_name ] = protein_sequence
gene_name_to_uniprot_id[ gene_name ] = uniprot_id
domains = get_domain_annotations( uniprot_id )
gene_name_to_domains[ gene_name ] = domains

################################################
uniprot_id = 'Q92614'
gene_name = 'MYO18A'
url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.fasta"

response = requests.get(url)

if response.status_code == 200:
    fasta_data = response.text
    # Parse the FASTA format and extract the sequence
    lines = fasta_data.splitlines()
    protein_sequence = ''.join(lines[1:])  # Join all lines except the header
gene_name_to_protein_seq[ gene_name ] = protein_sequence
gene_name_to_uniprot_id[ gene_name ] = uniprot_id
domains = get_domain_annotations( uniprot_id )
gene_name_to_domains[ gene_name ] = domains

################################################
uniprot_id = 'P98175'
gene_name = 'RBM10'
url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.fasta"

response = requests.get(url)

if response.status_code == 200:
    fasta_data = response.text
    # Parse the FASTA format and extract the sequence
    lines = fasta_data.splitlines()
    protein_sequence = ''.join(lines[1:])  # Join all lines except the header
gene_name_to_protein_seq[ gene_name ] = protein_sequence
gene_name_to_uniprot_id[ gene_name ] = uniprot_id
domains = get_domain_annotations( uniprot_id )
gene_name_to_domains[ gene_name ] = domains

################################################
uniprot_id = 'A0A0A6YYI9' # I think ?
gene_name = 'RBM14-RBM4'
url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.fasta"

response = requests.get(url)

if response.status_code == 200:
    fasta_data = response.text
    # Parse the FASTA format and extract the sequence
    lines = fasta_data.splitlines()
    protein_sequence = ''.join(lines[1:])  # Join all lines except the header
gene_name_to_protein_seq[ gene_name ] = protein_sequence
gene_name_to_uniprot_id[ gene_name ] = uniprot_id
domains = get_domain_annotations( uniprot_id )
gene_name_to_domains[ gene_name ] = domains

################################################
#omit RPL17-C18orf32 --> I am really not sure what protein this corresponds to

################################################
uniprot_id = 'P42166' # the 2 are just different isoforms, not sure which is "correct", pick alpha for now
gene_name = 'TMPO'
url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.fasta"

response = requests.get(url)

if response.status_code == 200:
    fasta_data = response.text
    # Parse the FASTA format and extract the sequence
    lines = fasta_data.splitlines()
    protein_sequence = ''.join(lines[1:])  # Join all lines except the header
gene_name_to_protein_seq[ gene_name ] = protein_sequence
gene_name_to_uniprot_id[ gene_name ] = uniprot_id
domains = get_domain_annotations( uniprot_id )
gene_name_to_domains[ gene_name ] = domains

In [ ]:
df['uniprot_id'] = df['Name'].map(gene_name_to_uniprot_id)
df['sequence'] = df['Name'].map(gene_name_to_protein_seq)

In [ ]:
import re

def normalize_domain(domain_name):
    # Removes trailing numbers and whitespace
    return re.sub(r'\s\d+.*$', '', domain_name).strip()

In [ ]:
# Gather domain names
unique_domains = set(
    normalize_domain(domain_name)
    for domains in gene_name_to_domains.values()
    for domain_name in domains
)
print(f"Total unique domains: {len(unique_domains)}")

In [ ]:
from collections import defaultdict

genes = gene_name_to_domains.keys()
sequences = gene_name_to_protein_seq.values()
domain_data = defaultdict(dict)

# Build stats per gene
for gene, sequence in zip(genes, sequences):
    seq_len = len(sequence)
    domain_stats = defaultdict(list)
    domain_string_map = defaultdict(lambda: ["0"] * seq_len)
    
    for raw_name, pos_dict in gene_name_to_domains[gene].items():
        norm_name = normalize_domain(raw_name)
        try:
            begin = int(pos_dict['begin'])
            end = int(pos_dict['end'])
            length = abs(end - begin)
            domain_stats[norm_name].append(length)

            for i in range(begin, end):
                    if 0 <= i < seq_len:
                        domain_string_map[norm_name][i] = "1"
                    
        except (ValueError, KeyError):
            continue
    
    for domain in unique_domains:
        lengths = domain_stats.get(domain, [])
        domain_data[gene][f"Number of {domain}"] = len(lengths)
        domain_data[gene][f"Average Size of {domain}"] = sum(lengths)/len(lengths) if lengths else 0
        domain_data[gene][f"Total Size of {domain}"] = sum(lengths)
        domain_data[gene][f"{domain} Map"] = "".join(domain_string_map[domain])

In [ ]:
# Convert domain_data to a DataFrame
domain_df = pd.DataFrame.from_dict(domain_data, orient="index")

# Merge with existing DataFrame
df = df.merge(domain_df, left_on="Name", right_index=True, how="left")

In [ ]:
def get_zinc_finger_annotations(uniprot_id):
    """Fetches Zinc Finger regions labeled 'ZN_FING' from UniProt's features API."""
    base_url = f"https://www.ebi.ac.uk/proteins/api/features/{uniprot_id}"
    response = requests.get(base_url, headers={"Accept": "application/json"})

    if response.ok:
        zinc_features = {}
        for feat in response.json().get("features", []):
            if feat.get("type") == "ZN_FING":
                desc = feat.get("description", "Zinc Finger")
                zinc_features[desc] = {
                    "begin": feat["begin"],
                    "end": feat["end"]
                }
        return zinc_features
    else:
        raise ValueError(f"Failed to fetch features for UniProt ID {uniprot_id}")

gene_name_to_fingers = {}
for index, row in df.iterrows():
    #gene_name = row['UNIQUE']  # Assuming "UNIQUE" is the gene name column
    #print( index, row['ID'] )
    gene_name = row['Name']
    ensembl_gene_id = row['ID'] # Assuming "ID" is the gene ID column
    #if ensembl_gene_id != 'ENSG00000231135': continue
    ensembl_gene_id_map = ''
    if ';' in ensembl_gene_id:
        for gid in ensembl_gene_id.split(';'):
            if gid in genemap_df['ensembl_gene_id'].tolist():
                ensembl_gene_id_map = gid
                break
                
    else:
        ensembl_gene_id_map = ensembl_gene_id

    if ensembl_gene_id_map == '':
        print( f"didn't find mapped id for {row['ID']}" )
    
    try:
        protein_sequence, uniprot_id = get_protein_sequence_mapped(gene_name=ensembl_gene_id_map)
        gene_name_to_protein_seq[ gene_name ] = protein_sequence
        gene_name_to_uniprot_id[ gene_name ] = uniprot_id
        if uniprot_id == '':
            gene_name_to_fingers[ gene_name ] = {}
        else:
            fingers = get_zinc_finger_annotations( uniprot_id )
            gene_name_to_fingers[ gene_name ] = fingers
        #print(f"Gene: {gene_name}, Ensembl ID: {ensembl_gene_id}, Protein Sequence: {protein_sequence[:50]}...")  # Print first 50 characters of the sequence
    except ValueError as e:
        print(f"Error for Gene: {gene_name}: {e}")
        
    if (index % 100) == 0: 
        print( index )

In [ ]:
# manually add the missing proteins
uniprot_id = 'P42771'
gene_name = 'CDKN2A'

gene_name_to_uniprot_id[ gene_name ] = uniprot_id
fingers = get_zinc_finger_annotations( uniprot_id )
gene_name_to_fingers[ gene_name ] = fingers

################################################
uniprot_id = 'Q9UPN3'
gene_name = 'MACF1'

gene_name_to_uniprot_id[ gene_name ] = uniprot_id
fingers = get_zinc_finger_annotations( uniprot_id )
gene_name_to_fingers[ gene_name ] = fingers

################################################
uniprot_id = 'Q92614'
gene_name = 'MYO18A'

gene_name_to_uniprot_id[ gene_name ] = uniprot_id
fingers = get_zinc_finger_annotations( uniprot_id )
gene_name_to_fingers[ gene_name ] = fingers

################################################
uniprot_id = 'P98175'
gene_name = 'RBM10'

gene_name_to_uniprot_id[ gene_name ] = uniprot_id
fingers = get_zinc_finger_annotations( uniprot_id )
gene_name_to_fingers[ gene_name ] = fingers

################################################
uniprot_id = 'A0A0A6YYI9' # I think ?
gene_name = 'RBM14-RBM4'

gene_name_to_uniprot_id[ gene_name ] = uniprot_id
fingers = get_zinc_finger_annotations( uniprot_id )
gene_name_to_fingers[ gene_name ] = fingers

################################################
#omit RPL17-C18orf32 --> I am really not sure what protein this corresponds to

################################################
uniprot_id = 'P42166' # the 2 are just different isoforms, not sure which is "correct", pick alpha for now
gene_name = 'TMPO'

gene_name_to_uniprot_id[ gene_name ] = uniprot_id
fingers = get_zinc_finger_annotations( uniprot_id )
gene_name_to_fingers[ gene_name ] = fingers

In [ ]:
# Gather zinc finger names
unique_fingers = set(
    normalize_domain(finger_name)
    for fingers in gene_name_to_fingers.values()
    for finger_name in fingers
)

print(f"Total unique Zinc Fingers: {len(unique_fingers)}")

In [ ]:
genes = gene_name_to_domains.keys()
sequences = gene_name_to_protein_seq.values()
finger_data = defaultdict(dict)

# Build stats per gene
for gene, sequence in zip(genes, sequences):
    seq_len = len(sequence)
    finger_stats = defaultdict(list)
    finger_string_map = defaultdict(lambda: ["0"] * seq_len)
    
    for raw_name, pos_dict in gene_name_to_fingers[gene].items():
        norm_name = normalize_domain(raw_name)
        try:
            begin = int(pos_dict['begin'])
            end = int(pos_dict['end'])
            length = abs(end - begin)
            finger_stats[norm_name].append(length)

            for i in range(begin, end):
                    if 0 <= i < seq_len:
                        finger_string_map[norm_name][i] = "1"
                    
        except (ValueError, KeyError):
            continue
    
    for finger in all_fingers:
        lengths = finger_stats.get(finger, [])
        finger_data[gene][f"Number of {finger}"] = len(lengths)
        finger_data[gene][f"Average Size of {finger}"] = sum(lengths)/len(lengths) if lengths else 0
        finger_data[gene][f"Total Size of {finger}"] = sum(lengths)
        finger_data[gene][f"{finger} Map"] = "".join(finger_string_map[finger])

In [ ]:
# Convert finger_data to a DataFrame
finger_df = pd.DataFrame.from_dict(finger_data, orient="index")

# Merge with existing DataFrame
df = df.merge(finger_df, left_on="Name", right_index=True, how="left")